In [ ]:
!pip install duckdb --pre --upgrade

In [2]:
sf =100

StatementMeta(, 8a782575-b5fc-487e-bec1-f4f4c2587530, 4, Finished, Available)

In [2]:
%%time

import duckdb
import pathlib
for x in range(0, sf) :
  con=duckdb.connect()
  con.sql('PRAGMA disable_progress_bar;SET preserve_insertion_order=false')
  con.sql(f"CALL dbgen(sf={sf} , children ={sf}, step = {x})") 
  for tbl in ['nation','region','customer','supplier','lineitem','orders','partsupp','part'] :
     pathlib.Path(f'/lakehouse/default/Files/{sf}/{tbl}').mkdir(parents=True, exist_ok=True) 
     con.sql(f"COPY (SELECT * FROM {tbl}) TO '/lakehouse/default/Files/{sf}/{tbl}/{x:03d}.parquet' ")
  con.close()

StatementMeta(, 2cdd069f-eb00-444e-bf73-d790ab9a82bc, 4, Finished, Available)

CPU times: user 9h 22min 10s, sys: 28min 45s, total: 9h 50min 56s
Wall time: 4h 21min 16s


In [1]:
spark.conf.set("spark.sql.parquet.vorder.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true")

StatementMeta(, dff08c24-ef19-4512-a421-b2b82a922c91, 3, Finished, Available)

In [3]:
from pyspark.sql.types import *
def loadFullDataFromSource(table_name):
    df = spark.read.parquet(f'Files/{sf}/' + table_name + '/*.parquet')
    df.write.mode("overwrite").format("delta").save("Tables/" + table_name)
full_tables = [
    'customer',
    'lineitem',
    'nation',
    'orders' ,
    'region',
    'partsupp',
    'supplier' ,
    'part'
    ]

for table in full_tables:
    loadFullDataFromSource(table)

StatementMeta(, 8a782575-b5fc-487e-bec1-f4f4c2587530, 5, Finished, Available)